In [1]:
from cracknuts import CrackerG1
import cracknuts as cn
from cracknuts.cracker import serial
import time
import random

In [2]:
g1d = CrackerG1('192.168.0.10')
g1d.connect(force_update_bin=True, force_write_default_config=True)

In [12]:
cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(cracker):
    cracker.nut_voltage_enable()
    cracker.nut_voltage(3.3)
    cracker.nut_clock_enable()
    cracker.nut_clock_freq('8M')
    cracker.uart_io_enable()
    cracker.osc_sample_clock('48m')
    cracker.osc_sample_length(sample_length)
    cracker.osc_trigger_source('N')
    cracker.osc_analog_gain('B', 10)
    cracker.osc_trigger_level(0)
    cracker.osc_trigger_mode('E')
    cracker.osc_trigger_edge('U')
    cracker.uart_config(baudrate=serial.Baudrate.BAUDRATE_115200, bytesize=serial.Bytesize.EIGHTBITS, parity=serial.Parity.PARITY_NONE, stopbits=serial.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)
    # g1d.glitch_vcc_config(level=1.5, wait=100, delay=100, repeat=10, count=200)  #####
    g1d.glitch_gnd_config(level=1, wait=100, delay=100, repeat=10, length=200)  #####

def do(cracker, count):
    print(count)
    # cracker.glitch_vcc_arm()  #######
    cracker.glitch_gnd_arm()  #######
    plaintext_data = random.randbytes(aes_data_len)
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    return {
        "plaintext": plaintext_data,
        "ciphertext": None if ret is None else ret[-aes_data_len:],
        "key": bytes.fromhex(aes_key)
        
    }


def finish(c):
    ...
    # print('optional behavior')


In [13]:
acq = cn.new_acquisition(g1d, init=init, do=do, finish=finish)

In [14]:
acq.test()

0
1


[WARNING] 2025-12-19 16:42:25,324 cracker_basic.send_and_receive:526 Command 0x0263 with payload 00 16 01 00 00 27 10 01 02 00 00 00 00 00 10 cd 11 60 95 52 6b 72 db 42 05 f8 d3 c8 2b af 0a received a non-OK response with status code 0x0002, and payload [00 00 00 00 00 08 6c 12 a4 82 65 3b 81 ed].


2
3


[WARNING] 2025-12-19 16:42:39,402 cracker_basic.send_and_receive:526 Command 0x0263 with payload 00 16 01 00 00 27 10 01 02 00 00 00 00 00 10 7d 97 0c b2 f7 cc b5 d5 d9 61 8b 02 ca 97 6d 4e received a non-OK response with status code 0x0002, and payload [80 00 00 00 00 00].


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [15]:
acq.stop()

In [ ]:
g1d.get_current_config().osc_sample_length